In [2]:
from joblib import load
import pandas as pd
import numpy as np
import os
import torch

In [3]:
front = '''
⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⢠⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀  ⠀⠀⠀⠀⠀⠀ ⠀⠸⣿⣇⠀⠀⠀⠀⠀⠀  ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀      ⠀   ⠀⢠⣠⣢⣦⣶⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⢀⣄⣤⣀⠀⠀⠈⣿⣿⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀  ⠀   ⠀⠀⠀⣶⣷⣾⣾⣿⡆⠀⠀⠀   ⠀⠀⠀⠙⠛⠛⠙⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀  ⠀⠀⠀⣢⡄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⣿⡿⣿⣿⣧⠀⠀⢺⣿⣿⣷⣿⡯⠀⠀⢀⣀⣀⠀⠀⠐⣨⣤⡀⠀⠀⠀   ⠀⠀⠀⠀⢽⡏⠁⢠⣿⠇⠀⠀⠀⠀⠀     ⠀⠀⠀⠀⢺⣿⣗⠀⠀⠀⠀⠀    ⠀ ⠀⠀⢸⣷⠀⠀⠀⠀⠀⠀⠀⠀
⠡⠈⠄⠅⡘⣿⣇⠄⣹⣿⠆⢂⢚⣿⣿⠉⠅⡂⠂⢬⣿⣿⣿⣷⡈⠄⢺⣿⣿⡆⠨⢐⠨⠠⢁⠂⢌⣵⣴⣾⣿⠂⡂⢂⠂⠅⡂⠅⠂⠅⡂⠂⠌⣿⢗⠐⡐⠨⢐⠠⢵⣿⡗⢐⠠⢁⠂⡹⣿⣅⣂⣅⠌⠄⠅⠌⡐
⠈⠌⠄⠡⠈⢿⣿⣿⣾⡟⠨⢀⠂⣿⣿⡕⢐⠠⠁⣿⣟⠨⠘⣿⣯⠠⠱⣿⣿⣿⣿⣷⡎⡐⠠⠈⠌⢿⠿⠿⡃⢂⢐⠠⠨⢐⠠⣁⢅⡂⡄⡡⣁⢂⡂⡂⠄⠅⡂⠌⣾⣿⡃⡐⠨⠐⡐⠸⣿⣿⠿⢿⠏⡂⠅⡂⡐
⠅⠌⠨⠠⢁⠂⠌⠩⢐⠠⢁⠂⠌⣺⣿⡯⢐⠨⢐⢸⣿⣿⣮⣿⠇⡂⠡⢿⣿⡇⠅⠌⡐⢠⣅⣅⣌⣄⣅⣢⣐⣔⣄⣢⡁⢂⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡷⠠⢁⢿⣿⣦⣦⣥⡡⠠⠡⣿⣿⢈⠐⡀⡂⠅⡐⡀
⠈⠌⠄⡁⡂⠌⠨⠐⣐⣼⣶⣾⣤⢈⢛⠑⠠⢈⢐⠠⢉⠛⡋⢑⠐⠠⠁⠍⢛⠠⢁⢂⢐⠈⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⡐⢐⢀⠂⠅⠌⠨⠈⠍⡉⠍⠩⢐⠨⠀⠍⢛⠛⡛⠛⠅⡂⡁⢿⣿⡐⢐⢀⠂⠅⡐⢐
⠌⠄⠡⢐⠠⠡⠨⠐⡈⢛⠋⣻⣿⠠⠐⡈⠌⠄⠂⠌⡐⠐⡐⢐⠨⠠⠡⢈⢐⠨⢀⢂⠐⠨⢐⢀⠂⢅⣍⡍⠨⢈⠐⡐⡀⢂⢂⠐⠨⠀⠅⣽⡇⠅⠂⠅⠌⡐⡈⠌⠨⢐⢀⠂⠅⡂⡐⠠⢙⠑⡐⡀⠂⠌⡐⢐⢀
⠀⠅⡁⢂⠂⠅⠨⠐⡐⡀⡂⣾⣗⣨⣴⢐⠈⠌⠨⢐⠠⢁⢂⠐⠠⠡⠈⠄⡂⠌⠄⡂⠌⡐⡀⠂⠌⠨⣿⣿⣇⠐⡐⡀⢂⢂⠐⡈⠌⠨⣸⣿⣧⣌⠨⠠⠡⠐⠠⠁⠅⡂⢐⠨⢐⠀⡂⠅⢂⢂⠐⠠⠁⠅⢂⢂⠐
⠁⠁⠂⠂⠈⠈⠠⠁⠐⠀⠂⠛⠟⠛⠑⠀⠌⠈⠐⠀⠌⠀⠂⠈⠄⠁⠁⠂⠂⠈⠐⠀⠁⠐⠀⠁⠁⠁⣿⣿⣿⠐⠀⠂⠂⠐⠐⠀⠁⠢⣿⠇⠙⢿⣷⣮⡀⠁⠁⠁⠂⠂⠂⠈⠐⠐⠀⠁⠂⠐⠈⠈⠈⠈⠀⠂⠂
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀     ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
                                            [사투리 분류 모델기]
'''

print(front)

print('이용할 분류기를 선택하세요')
print(''' 
1. 경상도 사투리 VS 표준말 
2. 전라도 사투리 VS 표준말
3. 제주도 사투리 VS 표준말
4. 5도 사투리 + 표준말 분류
'''
)


⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀⠀⢠⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀  ⠀⠀⠀⠀⠀⠀ ⠀⠸⣿⣇⠀⠀⠀⠀⠀⠀  ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀      ⠀   ⠀⢠⣠⣢⣦⣶⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⢀⣄⣤⣀⠀⠀⠈⣿⣿⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⠀  ⠀   ⠀⠀⠀⣶⣷⣾⣾⣿⡆⠀⠀⠀   ⠀⠀⠀⠙⠛⠛⠙⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀  ⠀⠀⠀⣢⡄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⣿⡿⣿⣿⣧⠀⠀⢺⣿⣿⣷⣿⡯⠀⠀⢀⣀⣀⠀⠀⠐⣨⣤⡀⠀⠀⠀   ⠀⠀⠀⠀⢽⡏⠁⢠⣿⠇⠀⠀⠀⠀⠀     ⠀⠀⠀⠀⢺⣿⣗⠀⠀⠀⠀⠀    ⠀ ⠀⠀⢸⣷⠀⠀⠀⠀⠀⠀⠀⠀
⠡⠈⠄⠅⡘⣿⣇⠄⣹⣿⠆⢂⢚⣿⣿⠉⠅⡂⠂⢬⣿⣿⣿⣷⡈⠄⢺⣿⣿⡆⠨⢐⠨⠠⢁⠂⢌⣵⣴⣾⣿⠂⡂⢂⠂⠅⡂⠅⠂⠅⡂⠂⠌⣿⢗⠐⡐⠨⢐⠠⢵⣿⡗⢐⠠⢁⠂⡹⣿⣅⣂⣅⠌⠄⠅⠌⡐
⠈⠌⠄⠡⠈⢿⣿⣿⣾⡟⠨⢀⠂⣿⣿⡕⢐⠠⠁⣿⣟⠨⠘⣿⣯⠠⠱⣿⣿⣿⣿⣷⡎⡐⠠⠈⠌⢿⠿⠿⡃⢂⢐⠠⠨⢐⠠⣁⢅⡂⡄⡡⣁⢂⡂⡂⠄⠅⡂⠌⣾⣿⡃⡐⠨⠐⡐⠸⣿⣿⠿⢿⠏⡂⠅⡂⡐
⠅⠌⠨⠠⢁⠂⠌⠩⢐⠠⢁⠂⠌⣺⣿⡯⢐⠨⢐⢸⣿⣿⣮⣿⠇⡂⠡⢿⣿⡇⠅⠌⡐⢠⣅⣅⣌⣄⣅⣢⣐⣔⣄⣢⡁⢂⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡷⠠⢁⢿⣿⣦⣦⣥⡡⠠⠡⣿⣿⢈⠐⡀⡂⠅⡐⡀
⠈⠌⠄⡁⡂⠌⠨⠐⣐⣼⣶⣾⣤⢈⢛⠑⠠⢈⢐⠠⢉⠛⡋⢑⠐⠠⠁⠍⢛⠠⢁⢂⢐⠈⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡏⡐⢐⢀⠂⠅⠌⠨⠈⠍⡉⠍⠩⢐⠨⠀⠍⢛⠛⡛⠛⠅⡂⡁⢿⣿⡐⢐⢀⠂⠅⡐⢐
⠌⠄⠡⢐⠠⠡⠨⠐⡈⢛⠋⣻⣿⠠⠐⡈⠌⠄⠂⠌⡐⠐⡐⢐⠨⠠⠡⢈⢐⠨⢀⢂⠐⠨⢐⢀⠂⢅⣍⡍⠨⢈⠐⡐⡀⢂⢂⠐⠨⠀⠅⣽⡇⠅⠂⠅⠌⡐⡈⠌⠨⢐⢀⠂⠅⡂⡐⠠⢙⠑⡐⡀⠂⠌⡐⢐⢀
⠀⠅⡁⢂⠂⠅⠨⠐⡐⡀⡂⣾⣗⣨⣴⢐⠈⠌⠨⢐⠠⢁⢂⠐⠠⠡⠈⠄⡂⠌⠄⡂⠌⡐⡀⠂⠌⠨⣿⣿⣇⠐⡐⡀⢂⢂⠐⡈⠌⠨⣸⣿⣧⣌⠨⠠⠡⠐⠠⠁⠅⡂⢐⠨⢐⠀⡂⠅⢂⢂⠐⠠⠁⠅⢂⢂⠐
⠁⠁⠂⠂⠈⠈⠠⠁⠐⠀⠂⠛⠟⠛⠑⠀⠌⠈⠐⠀⠌⠀⠂⠈⠄⠁⠁⠂⠂⠈⠐⠀⠁⠐⠀⠁⠁⠁⣿⣿⣿⠐⠀⠂⠂⠐⠐⠀⠁⠢⣿⠇⠙⢿⣷⣮⡀⠁⠁⠁⠂⠂⠂⠈⠐⠐⠀⠁⠂⠐⠈⠈⠈⠈⠀⠂⠂
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀     ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
                                         

In [ ]:
while True:
    model_num = int(input('분류기 번호를 입력하세요. :'))
    if model_num == 1:
        torch.load('model.model.pth')
        break
    elif model_num == 2 :
        torch.load('model.model.pth')
        break
    elif model_num == 3 :
        torch.load('model.model.pth')
        break
    elif model_num == 4 :
        torch.load('model.model.pth')
        break
    else:print('번호를 제대로 다시 입력해주세요')